In [4]:
import cv2, sys
from matplotlib import pyplot as plt
import numpy as np

In [5]:
img_path = 'trail_train_data\surface_img/cancer2.jpg'
image = cv2.imread(img_path)
#image = cv2.resize(image, (500, 500), interpolation = cv2.INTER_CUBIC)

image_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#image_gray = cv2.resize(image_gray, (500, 500), interpolation = cv2.INTER_CUBIC)




In [6]:
b,g,r = cv2.split(image)
image2 = cv2.merge([r,g,b])
 
plt.imshow(image2)
plt.xticks([])
plt.yticks([])
plt.show()

ValueError: not enough values to unpack (expected 3, got 0)

In [ ]:
#cv2.imshow('image', image)
#cv2.imshow('image_gray', image_gray)
# 
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
blur = cv2.GaussianBlur(image_gray, ksize=(3,3), sigmaX=0)
ret, thresh1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)

In [ ]:
blur = cv2.GaussianBlur(image_gray, ksize=(5,5), sigmaX=0)
ret, thresh1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)


In [ ]:
edged = cv2.Canny(blur, 10, 250)
#cv2.imshow('Edged', edged)
#cv2.waitKey(0)

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)

#cv2.imshow('closed', closed)
#cv2.waitKey(0)


In [ ]:
contours, _ = cv2.findContours(closed.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
total = 0


In [ ]:
# 외곽선 그리는 용도. 이미지에 그리기 때문에 이 코드 적용하면 원래 이미지에
# 초록색 선 생김
#contours_image = cv2.drawContours(image, contours, -1, (0,255,0), 3)
#cv2.imshow('contours_image', contours_image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


In [ ]:
contours_xy = np.array(contours)
contours_xy.shape

# x의 min과 max 찾기
x_min, x_max = 0,0
value = list()
for i in range(len(contours_xy)):
    for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][0]) #네번째 괄호가 0일때 x의 값
        x_min = min(value)
        x_max = max(value)
print(x_min)
print(x_max)
 
# y의 min과 max 찾기
y_min, y_max = 0,0
value = list()
for i in range(len(contours_xy)):
    for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][1]) #네번째 괄호가 0일때 x의 값
        y_min = min(value)
        y_max = max(value)
print(y_min)
print(y_max)


0
203
0
246


C:\Users\bodle\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
# image trim 하기
x = x_min
y = y_min
w = x_max-x_min
h = y_max-y_min

In [ ]:
img_trim = image[y:y+h, x:x+w]
cv2.imwrite('org_trim.jpg', img_trim)
org_image = cv2.imread('org_trim.jpg')

In [ ]:
# 마우스로 관심영역 지정 및 표시, 저장 (roi_crop_mouse.py)

import cv2
import numpy as np

isDragging = False                      # 마우스 드래그 상태 저장 
x0, y0, w, h = -1,-1,-1,-1              # 영역 선택 좌표 저장
blue, red = (255,0,0),(0,0,255)         # 색상 값 

def onMouse(event,x,y,flags,param):     # 마우스 이벤트 핸들 함수  ---①
    global isDragging, x0, y0, img      # 전역변수 참조
    if event == cv2.EVENT_LBUTTONDOWN:  # 왼쪽 마우스 버튼 다운, 드래그 시작 ---②
        isDragging = True
        x0 = x
        y0 = y
    elif event == cv2.EVENT_MOUSEMOVE:  # 마우스 움직임 ---③
        if isDragging:                  # 드래그 진행 중
            img_draw = img.copy()       # 사각형 그림 표현을 위한 이미지 복제
            cv2.rectangle(img_draw, (x0, y0), (x, y), blue, 2) # 드래그 진행 영역 표시
            cv2.imshow('img', img_draw) # 사각형 표시된 그림 화면 출력
    elif event == cv2.EVENT_LBUTTONUP:  # 왼쪽 마우스 버튼 업 ---④
        if isDragging:                  # 드래그 중지
            isDragging = False          
            w = x - x0                  # 드래그 영역 폭 계산
            h = y - y0                  # 드래그 영역 높이 계산
            print("x:%d, y:%d, w:%d, h:%d" % (x0, y0, w, h))
            if w > 0 and h > 0:         # 폭과 높이가 양수이면 드래그 방향이 옳음 ---⑤
                img_draw = img.copy()   # 선택 영역에 사각형 그림을 표시할 이미지 복제
                # 선택 영역에 빨간 사각형 표시
                cv2.rectangle(img_draw, (x0, y0), (x, y), red, 2) 
                cv2.imshow('img', img_draw) # 빨간 사각형 그려진 이미지 화면 출력
                roi = img[y0:y0+h, x0:x0+w] # 원본 이미지에서 선택 영영만 ROI로 지정 ---⑥
                cv2.imshow('cropped', roi)  # ROI 지정 영역을 새창으로 표시
                cv2.moveWindow('cropped', 0, 0) # 새창을 화면 좌측 상단에 이동
                cv2.imwrite('./cropped.jpg', roi)   # ROI 영역만 파일로 저장 ---⑦
                print("croped.")
            else:
                cv2.imshow('img', img)  # 드래그 방향이 잘못된 경우 사각형 그림ㅇㅣ 없는 원본 이미지 출력
                print("좌측 상단에서 우측 하단으로 영역을 드래그 하세요.")

img = cv2.imread('org_trim.jpg')
cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse) # 마우스 이벤트 등록 ---⑧
cv2.waitKey()
cv2.destroyAllWindows()                                    

x:43, y:66, w:113, h:101
croped.
x:65, y:189, w:54, h:32
croped.
x:145, y:222, w:-86, h:-185
좌측 상단에서 우측 하단으로 영역을 드래그 하세요.
x:24, y:50, w:175, h:145
croped.
x:53, y:17, w:95, h:201
croped.
x:72, y:3, w:38, h:377
croped.
x:20, y:69, w:38, h:46
croped.
x:75, y:82, w:27, h:45
croped.
x:137, y:105, w:48, h:38
croped.
x:185, y:146, w:5, h:34
croped.
x:151, y:68, w:-93, h:58
좌측 상단에서 우측 하단으로 영역을 드래그 하세요.
x:61, y:127, w:2, h:4
croped.
x:150, y:76, w:-69, h:64
좌측 상단에서 우측 하단으로 영역을 드래그 하세요.
x:145, y:70, w:-60, h:69
좌측 상단에서 우측 하단으로 영역을 드래그 하세요.
x:141, y:72, w:-57, h:50
좌측 상단에서 우측 하단으로 영역을 드래그 하세요.
x:88, y:66, w:72, h:80
croped.
x:51, y:70, w:87, h:77
croped.
x:34, y:52, w:106, h:90
croped.
x:50, y:62, w:77, h:77
croped.
x:41, y:63, w:92, h:70
croped.
x:60, y:75, w:71, h:76
croped.
x:170, y:85, w:-99, h:70
좌측 상단에서 우측 하단으로 영역을 드래그 하세요.
x:130, y:62, w:-62, h:98
좌측 상단에서 우측 하단으로 영역을 드래그 하세요.
x:163, y:79, w:-131, h:108
좌측 상단에서 우측 하단으로 영역을 드래그 하세요.
x:44, y:72, w:72, h:96
croped.
x:58, y:95, w:118, h:108
cr

KeyboardInterrupt: 